In [36]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 100)

## Import Master Data Frames

In [44]:
sens = pd.read_csv('/Users/Alexz/CodeMaster/capstone/capstone_repo/data/senate_races_cleaned.csv')
govs = pd.read_csv('/Users/Alexz/CodeMaster/capstone/capstone_repo/data/governors_races_cleaned.csv')
this_year = pd.read_csv('/Users/Alexz/CodeMaster/capstone/capstone_repo/data/2018_data.csv')

sens.drop('Unnamed: 0', inplace=True, axis=1)
govs.drop('Unnamed: 0', inplace=True, axis=1)

In [45]:
print(this_year.shape)
this_year.head()

(35, 13)


,race_id,office,loc_date,state,abbrev,year,winner,rival,pred_GOP,pred_DEM,unopposed,inc_GOP_running,inc_DEM_running
0,2018_AZ_sen,AZ_sen,AZ_2018,Arizona,AZ,2018,TBD,TBD,1,0,0,0,0
1,2018_CA_sen,CA_sen,CA_2018,California,CA,2018,TBD,TBD,0,1,0,0,1
2,2018_CT_sen,CT_sen,CT_2018,Connecticut,CT,2018,TBD,TBD,0,1,0,0,1
3,2018_DE_sen,DE_sen,DE_2018,Delaware,DE,2018,TBD,TBD,0,1,0,0,1
4,2018_FL_sen,FL_sen,FL_2018,Florida,FL,2018,TBD,TBD,0,1,0,0,1


In [46]:
sens.head()

,race_id,office,loc_date_id,state,abbrev,year,GOP_win,winner,rival,pred_GOP,pred_DEM,unopposed,inc_GOP_running,inc_DEM_running
0,AK_sen_1978,AK_sen,AK_1978,Alaska,AK,1978,1,Ted Stevens,Donald W Hobbs,1,0,0,1,0
1,AK_sen_1980,AK_sen,AK_1980,Alaska,AK,1980,1,Frank Murkowski,Clark Gruening,0,1,0,0,0
2,AK_sen_1984,AK_sen,AK_1984,Alaska,AK,1984,1,Ted Stevens,John E Havelock,1,0,0,1,0
3,AK_sen_1986,AK_sen,AK_1986,Alaska,AK,1986,1,Frank Murkowski,Glenn Olds,1,0,0,1,0
4,AK_sen_1990,AK_sen,AK_1990,Alaska,AK,1990,1,Ted Stevens,Michael Beasley,1,0,0,1,0


In [47]:
sen_states = sens.abbrev.unique()
gov_states = govs.abbrev.unique()
print(len(sen_states))
print(len(gov_states))
print(sen_states)
print(this_year.shape)

50
50
['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DE' 'FL' 'GA' 'HI' 'IA' 'ID' 'IL'
 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT' 'NC' 'ND'
 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD' 'TN'
 'TX' 'UT' 'VA' 'VT' 'WA' 'WI' 'WV' 'WY']
(35, 13)


## Join with Pres Approval Data

#### Import and Join

In [48]:
approval = pd.read_csv('/Users/Alexz/CodeMaster/capstone/capstone_repo/data/pres_approval.csv')
approval.head()

,president,pres_party,prez_GOP,month,year,month_year,approve,disapprove,unsure
0,Ford,R,1,10,1975,10_1975,44,43,12
1,Ford,R,1,10,1975,10_1975,46,40,13
2,Ford,R,1,10,1975,10_1975,47,36,16
3,Ford,R,1,10,1974,10_1974,53,28,17
4,Ford,R,1,10,1974,10_1974,52,28,19


In [49]:
approval_join = approval.groupby('month_year')[['prez_GOP','approve', 'disapprove', 'unsure', 'year']].mean()
approval_join.reset_index(drop=True, inplace=True)
approval_join.head(10)

,prez_GOP,approve,disapprove,unsure,year
0,1.0,52.500000,28.000000,18.000000,1974.0
1,1.0,45.666667,39.666667,13.666667,1975.0
2,0.0,53.000000,29.333333,16.333333,1977.0
3,0.0,49.000000,36.000000,14.000000,1978.0
4,0.0,30.000000,55.666667,12.333333,1979.0
5,1.0,41.000000,47.000000,10.000000,1982.0
6,1.0,45.000000,43.000000,10.000000,1983.0
7,1.0,55.500000,33.500000,9.000000,1984.0
8,1.0,62.000000,29.000000,8.000000,1985.0
9,1.0,64.000000,28.000000,7.000000,1986.0


In [50]:
govs_with_approval = pd.merge(govs, approval_join, on='year')
sens_with_approval = pd.merge(sens, approval_join, on='year')
this_year_with_approval = pd.merge(this_year, approval_join, on='year')

In [51]:
print(this_year_with_approval.shape)

(35, 17)


#### Engineer Features

In [52]:
this_year_with_approval.head()

,race_id,office,loc_date,state,abbrev,year,winner,rival,pred_GOP,pred_DEM,unopposed,inc_GOP_running,inc_DEM_running,prez_GOP,approve,disapprove,unsure
0,2018_AZ_sen,AZ_sen,AZ_2018,Arizona,AZ,2018,TBD,TBD,1,0,0,0,0,1.0,43.5,52.0,4.5
1,2018_CA_sen,CA_sen,CA_2018,California,CA,2018,TBD,TBD,0,1,0,0,1,1.0,43.5,52.0,4.5
2,2018_CT_sen,CT_sen,CT_2018,Connecticut,CT,2018,TBD,TBD,0,1,0,0,1,1.0,43.5,52.0,4.5
3,2018_DE_sen,DE_sen,DE_2018,Delaware,DE,2018,TBD,TBD,0,1,0,0,1,1.0,43.5,52.0,4.5
4,2018_FL_sen,FL_sen,FL_2018,Florida,FL,2018,TBD,TBD,0,1,0,0,1,1.0,43.5,52.0,4.5


In [53]:
sens_with_approval.head()

,race_id,office,loc_date_id,state,abbrev,year,GOP_win,winner,rival,pred_GOP,pred_DEM,unopposed,inc_GOP_running,inc_DEM_running,prez_GOP,approve,disapprove,unsure
0,AK_sen_1978,AK_sen,AK_1978,Alaska,AK,1978,1,Ted Stevens,Donald W Hobbs,1,0,0,1,0,0.0,49.0,36.0,14.0
1,AL_sen_1978,AL_sen,AL_1978,Alabama,AL,1978,0,Howell Heflin,Jerome B Couch,0,1,0,0,0,0.0,49.0,36.0,14.0
2,AL_sen2_1978,AL_sen,AL_1978,Alabama,AL,1978,0,Donald W Stewart,James D Martin,0,1,0,0,0,0.0,49.0,36.0,14.0
3,AR_sen_1978,AR_sen,AR_1978,Arkansas,AR,1978,0,David Pryor,Tom Kelly,0,1,0,0,0,0.0,49.0,36.0,14.0
4,CO_sen_1978,CO_sen,CO_1978,Colorado,CO,1978,1,William L Armstrong,Floyd K Haskell,0,1,0,0,1,0.0,49.0,36.0,14.0


In [54]:
def approval_engineer(df):
    n_rows = df.shape[0]
    
    df['approval_effects_GOP'] = 0
    df['approval_effects_DEM'] = 0
    df['approval_gap'] = df['approve'] - df['disapprove']
    
    for i in range(n_rows):
        if df.loc[i, 'prez_GOP'] == 1:
            df.loc[i, 'approval_effects_GOP'] = df.loc[i, 'approval_gap']
        else:
            df.loc[i, 'approval_effects_DEM'] = df.loc[i, 'approval_gap']
        
    df.drop(['approval_gap', 'approve', 'disapprove', 'unsure'], inplace=True, axis=1)
    return df

In [55]:
govs_with_approval = approval_engineer(govs_with_approval)
sens_with_approval = approval_engineer(sens_with_approval)
this_year_with_approval = approval_engineer(this_year_with_approval)

In [56]:
this_year_with_approval.head()

,race_id,office,loc_date,state,abbrev,year,winner,rival,pred_GOP,pred_DEM,unopposed,inc_GOP_running,inc_DEM_running,prez_GOP,approval_effects_GOP,approval_effects_DEM
0,2018_AZ_sen,AZ_sen,AZ_2018,Arizona,AZ,2018,TBD,TBD,1,0,0,0,0,1.0,-8.5,0
1,2018_CA_sen,CA_sen,CA_2018,California,CA,2018,TBD,TBD,0,1,0,0,1,1.0,-8.5,0
2,2018_CT_sen,CT_sen,CT_2018,Connecticut,CT,2018,TBD,TBD,0,1,0,0,1,1.0,-8.5,0
3,2018_DE_sen,DE_sen,DE_2018,Delaware,DE,2018,TBD,TBD,0,1,0,0,1,1.0,-8.5,0
4,2018_FL_sen,FL_sen,FL_2018,Florida,FL,2018,TBD,TBD,0,1,0,0,1,1.0,-8.5,0


In [57]:
print(this_year.shape)

(35, 13)


In [58]:
# reset to default names
sens = sens_with_approval
govs = govs_with_approval
this_year = this_year_with_approval

In [59]:
sens.head()

,race_id,office,loc_date_id,state,abbrev,year,GOP_win,winner,rival,pred_GOP,pred_DEM,unopposed,inc_GOP_running,inc_DEM_running,prez_GOP,approval_effects_GOP,approval_effects_DEM
0,AK_sen_1978,AK_sen,AK_1978,Alaska,AK,1978,1,Ted Stevens,Donald W Hobbs,1,0,0,1,0,0.0,0.0,13.0
1,AL_sen_1978,AL_sen,AL_1978,Alabama,AL,1978,0,Howell Heflin,Jerome B Couch,0,1,0,0,0,0.0,0.0,13.0
2,AL_sen2_1978,AL_sen,AL_1978,Alabama,AL,1978,0,Donald W Stewart,James D Martin,0,1,0,0,0,0.0,0.0,13.0
3,AR_sen_1978,AR_sen,AR_1978,Arkansas,AR,1978,0,David Pryor,Tom Kelly,0,1,0,0,0,0.0,0.0,13.0
4,CO_sen_1978,CO_sen,CO_1978,Colorado,CO,1978,1,William L Armstrong,Floyd K Haskell,0,1,0,0,1,0.0,0.0,13.0


## Join with...

#### Import

In [60]:
state_unemp = pd.read_csv('/Users/Alexz/CodeMaster/capstone/capstone_repo/data/state_unemployment.csv')
nat_unemp = pd.read_csv('/Users/Alexz/CodeMaster/capstone/capstone_repo/data/national_unemployment.csv')

In [61]:
state_unemp.tail()

,state_year,unemployment_rate_state_oct
2146,MS_2018,4.8
2147,LA_2018,5.0
2148,WV_2018,5.2
2149,DC_2018,5.7
2150,AL_2018,6.5


In [62]:
nat_unemp.tail()

,month,year,unemployment_rate_national_oct
50,10,2014,5.7
51,10,2015,5.0
52,10,2016,4.9
53,10,2017,4.1
54,10,2018,3.7


In [63]:
this_year.head()

,race_id,office,loc_date,state,abbrev,year,winner,rival,pred_GOP,pred_DEM,unopposed,inc_GOP_running,inc_DEM_running,prez_GOP,approval_effects_GOP,approval_effects_DEM
0,2018_AZ_sen,AZ_sen,AZ_2018,Arizona,AZ,2018,TBD,TBD,1,0,0,0,0,1.0,-8.5,0
1,2018_CA_sen,CA_sen,CA_2018,California,CA,2018,TBD,TBD,0,1,0,0,1,1.0,-8.5,0
2,2018_CT_sen,CT_sen,CT_2018,Connecticut,CT,2018,TBD,TBD,0,1,0,0,1,1.0,-8.5,0
3,2018_DE_sen,DE_sen,DE_2018,Delaware,DE,2018,TBD,TBD,0,1,0,0,1,1.0,-8.5,0
4,2018_FL_sen,FL_sen,FL_2018,Florida,FL,2018,TBD,TBD,0,1,0,0,1,1.0,-8.5,0


In [64]:
sens_w_stateunemp = pd.merge(sens, state_unemp, left_on='loc_date_id', right_on = 'state_year')
sens_w_natunemp = pd.merge(sens_w_stateunemp, nat_unemp, on='year')

govs_w_stateunemp = pd.merge(govs, state_unemp, left_on='loc_date_id', right_on = 'state_year')
govs_w_natunemp = pd.merge(govs_w_stateunemp, nat_unemp, on='year')

this_year_w_stateunemp = pd.merge(this_year, state_unemp, left_on='loc_date', right_on = 'state_year')
this_year_w_natunemp = pd.merge(this_year_w_stateunemp, nat_unemp, on='year')

In [65]:
sens_w_natunemp.head()

,race_id,office,loc_date_id,state,abbrev,year,GOP_win,winner,rival,pred_GOP,pred_DEM,unopposed,inc_GOP_running,inc_DEM_running,prez_GOP,approval_effects_GOP,approval_effects_DEM,state_year,unemployment_rate_state_oct,month,unemployment_rate_national_oct
0,AK_sen_1978,AK_sen,AK_1978,Alaska,AK,1978,1,Ted Stevens,Donald W Hobbs,1,0,0,1,0,0.0,0.0,13.0,AK_1978,10.2,10,5.8
1,AL_sen_1978,AL_sen,AL_1978,Alabama,AL,1978,0,Howell Heflin,Jerome B Couch,0,1,0,0,0,0.0,0.0,13.0,AL_1978,6.5,10,5.8
2,AL_sen2_1978,AL_sen,AL_1978,Alabama,AL,1978,0,Donald W Stewart,James D Martin,0,1,0,0,0,0.0,0.0,13.0,AL_1978,6.5,10,5.8
3,AR_sen_1978,AR_sen,AR_1978,Arkansas,AR,1978,0,David Pryor,Tom Kelly,0,1,0,0,0,0.0,0.0,13.0,AR_1978,6.4,10,5.8
4,CO_sen_1978,CO_sen,CO_1978,Colorado,CO,1978,1,William L Armstrong,Floyd K Haskell,0,1,0,0,1,0.0,0.0,13.0,CO_1978,5.1,10,5.8


In [66]:
this_year_w_natunemp.head()

,race_id,office,loc_date,state,abbrev,year,winner,rival,pred_GOP,pred_DEM,unopposed,inc_GOP_running,inc_DEM_running,prez_GOP,approval_effects_GOP,approval_effects_DEM,state_year,unemployment_rate_state_oct,month,unemployment_rate_national_oct
0,2018_AZ_sen,AZ_sen,AZ_2018,Arizona,AZ,2018,TBD,TBD,1,0,0,0,0,1.0,-8.5,0,AZ_2018,4.7,10,3.7
1,2018_CA_sen,CA_sen,CA_2018,California,CA,2018,TBD,TBD,0,1,0,0,1,1.0,-8.5,0,CA_2018,4.1,10,3.7
2,2018_CT_sen,CT_sen,CT_2018,Connecticut,CT,2018,TBD,TBD,0,1,0,0,1,1.0,-8.5,0,CT_2018,4.2,10,3.7
3,2018_DE_sen,DE_sen,DE_2018,Delaware,DE,2018,TBD,TBD,0,1,0,0,1,1.0,-8.5,0,DE_2018,4.0,10,3.7
4,2018_FL_sen,FL_sen,FL_2018,Florida,FL,2018,TBD,TBD,0,1,0,0,1,1.0,-8.5,0,FL_2018,3.5,10,3.7


In [67]:
print(this_year_w_natunemp.shape)

(35, 20)


#### Feature Engineering

In [68]:
def unemployment_engineering(df):
    n_rows = df.shape[0]
    df.drop(['state_year', 'month'], inplace=True, axis=1)
    
    df['nat_UR_effects_GOP'] = 0
    df['nat_UR_effects_DEM'] = 0
    df['state_UR_effects_GOP'] = 0
    df['state_UR_effects_DEM'] = 0
    
    for i in range(n_rows):
        if df.loc[i, 'prez_GOP'] == 1:
            df.loc[i, 'nat_UR_effects_GOP'] = df.loc[i, 'unemployment_rate_national_oct']
            df.loc[i, 'state_UR_effects_GOP'] = df.loc[i, 'unemployment_rate_state_oct']
        else:
            df.loc[i, 'nat_UR_effects_DEM'] = df.loc[i, 'unemployment_rate_national_oct']
            df.loc[i, 'state_UR_effects_DEM'] = df.loc[i, 'unemployment_rate_state_oct']
    
    df.drop(['unemployment_rate_national_oct', 'unemployment_rate_state_oct'], axis=1, inplace=True)
    
    return df

#### Update Values

In [69]:
sens = unemployment_engineering(sens_w_natunemp)
govs = unemployment_engineering(govs_w_natunemp)
this_year = unemployment_engineering(this_year_w_natunemp)

## Join Census Data

#### Import

In [70]:
census = pd.read_csv('/Users/Alexz/CodeMaster/capstone/capstone_repo/data/census_final.csv')
census.drop(['year', 'state', 'abbrev', 'combo1'], axis=1, inplace=True)

In [71]:
census.head()

,combo2,total_population,total_male,total_female,age_under18,age_18to29,age_30to59,age_60over,race_white_nh,race_black,race_natamer,race_asian,race_hispanic,percent_male,percent_female,percent_under18,percent_age_18to29,percent_age_30to59,percent_age_60over,percent_race_white,percent_race_black,percent_race_natamer,percent_race_asian,percent_race_hispanic
0,AL_1970,3444165,1661941,1782224,1233520,619880,1115478,475287,2494983,903467,2443,4424,38848,0.482538,0.517462,0.358148,0.179980,0.323875,0.137998,0.724409,0.262318,0.000709,0.001284,0.011279
1,AL_1971,3489137,1682900,1806237,1226393,640938,1133137,488670,2529484,912749,3129,5048,38293,0.482326,0.517674,0.351489,0.183695,0.324761,0.140055,0.724960,0.261597,0.000897,0.001447,0.010975
2,AL_1972,3534110,1703860,1830250,1219266,661997,1150795,502052,2563985,922030,3815,5671,37738,0.482119,0.517881,0.344999,0.187316,0.325625,0.142059,0.725497,0.260895,0.001079,0.001605,0.010678
3,AL_1973,3579082,1724819,1854263,1212138,683055,1168454,515435,2598485,931312,4501,6295,37183,0.481917,0.518083,0.338673,0.190846,0.326468,0.144013,0.726020,0.260210,0.001258,0.001759,0.010389
4,AL_1974,3624054,1745778,1878276,1205011,704113,1186112,528818,2632986,940593,5187,6918,36628,0.481720,0.518280,0.332504,0.194289,0.327289,0.145919,0.726531,0.259542,0.001431,0.001909,0.010107


In [72]:
sens.head()

,race_id,office,loc_date_id,state,abbrev,year,GOP_win,winner,rival,pred_GOP,pred_DEM,unopposed,inc_GOP_running,inc_DEM_running,prez_GOP,approval_effects_GOP,approval_effects_DEM,nat_UR_effects_GOP,nat_UR_effects_DEM,state_UR_effects_GOP,state_UR_effects_DEM
0,AK_sen_1978,AK_sen,AK_1978,Alaska,AK,1978,1,Ted Stevens,Donald W Hobbs,1,0,0,1,0,0.0,0.0,13.0,0.0,5.8,0.0,10.2
1,AL_sen_1978,AL_sen,AL_1978,Alabama,AL,1978,0,Howell Heflin,Jerome B Couch,0,1,0,0,0,0.0,0.0,13.0,0.0,5.8,0.0,6.5
2,AL_sen2_1978,AL_sen,AL_1978,Alabama,AL,1978,0,Donald W Stewart,James D Martin,0,1,0,0,0,0.0,0.0,13.0,0.0,5.8,0.0,6.5
3,AR_sen_1978,AR_sen,AR_1978,Arkansas,AR,1978,0,David Pryor,Tom Kelly,0,1,0,0,0,0.0,0.0,13.0,0.0,5.8,0.0,6.4
4,CO_sen_1978,CO_sen,CO_1978,Colorado,CO,1978,1,William L Armstrong,Floyd K Haskell,0,1,0,0,1,0.0,0.0,13.0,0.0,5.8,0.0,5.1


In [73]:
sens_w_census = pd.merge(sens, census, left_on='loc_date_id', right_on='combo2')
govs_w_census = pd.merge(govs, census, left_on='loc_date_id', right_on='combo2')
this_year_w_census = pd.merge(this_year, census, left_on='loc_date', right_on='combo2')

sens_w_census.drop(['combo2', 'total_population'], axis =1, inplace=True)
govs_w_census.drop(['combo2', 'total_population'], axis =1, inplace=True)
this_year_w_census.drop(['combo2', 'total_population'], axis =1, inplace=True)



In [74]:
this_year_w_census.head()

,race_id,office,loc_date,state,abbrev,year,winner,rival,pred_GOP,pred_DEM,unopposed,inc_GOP_running,inc_DEM_running,prez_GOP,approval_effects_GOP,approval_effects_DEM,nat_UR_effects_GOP,nat_UR_effects_DEM,state_UR_effects_GOP,state_UR_effects_DEM,total_male,total_female,age_under18,age_18to29,age_30to59,age_60over,race_white_nh,race_black,race_natamer,race_asian,race_hispanic,percent_male,percent_female,percent_under18,percent_age_18to29,percent_age_30to59,percent_age_60over,percent_race_white,percent_race_black,percent_race_natamer,percent_race_asian,percent_race_hispanic
0,2018_AZ_sen,AZ_sen,AZ_2018,Arizona,AZ,2018,TBD,TBD,1,0,0,0,0,1.0,-8.5,0,3.7,0,4.7,0,3667636,3733489,1838668,1221211,4001177,1521795,2927154,339116,329049,1155312,2374775,0.495551,0.504449,0.248431,0.165003,0.540617,0.205617,0.395501,0.045820,0.044459,0.156100,0.320867
1,2018_CA_sen,CA_sen,CA_2018,California,CA,2018,TBD,TBD,0,1,0,0,1,1.0,-8.5,0,3.7,0,4.1,0,19832180,20127622,9331209,7273591,23576327,7147681,6029585,2327224,386365,12783605,16451449,0.496303,0.503697,0.233515,0.182023,0.590001,0.178872,0.150891,0.058239,0.009669,0.319912,0.411700
2,2018_CT_sen,CT_sen,CT_2018,Connecticut,CT,2018,TBD,TBD,0,1,0,0,1,1.0,-8.5,0,3.7,0,4.2,0,1811850,1897073,797277,595006,2122601,796269,2159956,404258,12550,419122,606098,0.488511,0.511489,0.214962,0.160426,0.572296,0.214690,0.582367,0.108996,0.003384,0.113004,0.163416
3,2018_DE_sen,DE_sen,DE_2018,Delaware,DE,2018,TBD,TBD,0,1,0,0,1,1.0,-8.5,0,3.7,0,4.0,0,478457,510944,214707,166077,557904,221162,543716,224732,5341,81125,101976,0.483582,0.516418,0.217007,0.167856,0.563881,0.223531,0.549541,0.227139,0.005398,0.081994,0.103068
4,2018_FL_sen,FL_sen,FL_2018,Florida,FL,2018,TBD,TBD,0,1,0,0,1,1.0,-8.5,0,3.7,0,3.5,0,10302667,10753789,4286692,3393223,11557300,5074659,9967789,3531348,85792,1465261,5456679,0.489288,0.510712,0.203581,0.161149,0.548872,0.241003,0.473384,0.167709,0.004074,0.069587,0.259145


In [75]:
print(this_year.shape)

(35, 20)


In [76]:
govs = govs_w_census
sens = sens_w_census
this_year = this_year_w_census

In [77]:
govs.to_csv('/Users/Alexz/CodeMaster/capstone/capstone_repo/data/govs_final.csv')
sens.to_csv('/Users/Alexz/CodeMaster/capstone/capstone_repo/data/sens_final.csv')
this_year.to_csv('/Users/Alexz/CodeMaster/capstone/capstone_repo/data/2018_final.csv')